# RDDの操作を設定する

[Introduction to Spark with Python, by Jose A. Dianes](https://github.com/jadianes/spark-py-notebooks)

Sparkは、RDD自体が適切に設定されていない場合でも、組合や交差点などの数学的なセットでの操作の多くをサポートしています。 これらの操作は、操作されているRDDが同じタイプであることを必要とすることに注意することが重要である。


セット操作は、期待どおりに動作するので、理解するのが非常に簡単です。 唯一の考慮事項は、RDDが実際のセットではないという事実から来ており、したがってRDDの組合などの操作は重複を除去しない。 このノートでは、減算、区別、およびデカルトを簡単に見ていきます。

## データの取得とRDDの作成

最初のノートブックで行ったように、KDD Cup 1999に提供された縮小データセット（10％）を使用し、約50万のネットワークインタラクションが含まれます。このファイルは、ローカルでダウンロードするGzipファイルとして提供されています。


In [1]:
import urllib
f = urllib.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

In [1]:
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

## subtractを使用して攻撃のやりとりを取得する

説明の目的のために、以前の分析からの非攻撃（正常）の相互作用を伴うRDDをすでに持っていると想像してください。 

In [9]:
normal_raw_data = raw_data.filter(lambda x: "normal." in x)
normal_raw_data.take(2)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.']

以下のように、元のフィルタリングされていないRDDから通常のものを減算することによって、攻撃相互作用を得ることができる。

In [10]:
attack_raw_data = raw_data.subtract(normal_raw_data)
attack_raw_data.take(2)

['305,tcp,telnet,SF,1735,2766,0,0,0,3,0,1,2,1,0,0,1,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,2,4,1.00,0.00,0.50,0.50,0.00,0.00,0.00,0.00,buffer_overflow.',
 '0,tcp,telnet,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,5,0.83,1.00,0.00,0.00,0.83,0.33,0.00,5,6,1.00,0.00,0.20,0.33,1.00,0.83,0.00,0.00,neptune.']

私たちの結果を確認するためにいくつかのカウントをしましょう。

In [5]:
from time import time

# count all
t0 = time()
raw_data_count = raw_data.count()
tt = time() - t0
print("All count in {} secs".format(round(tt,3)))

All count in 5.812 secs


In [6]:
# count normal
t0 = time()
normal_raw_data_count = normal_raw_data.count()
tt = time() - t0
print("Normal count in {} secs".format(round(tt,3)))

Normal count in 5.344 secs


In [7]:
# count attacks
t0 = time()
attack_raw_data_count = attack_raw_data.count()
tt = time() - t0
print("Attack count in {} secs".format(round(tt,3)))
attack_raw_data_count.take(2)

Attack count in 15.654 secs


In [8]:
print("There are {} normal interactions and {} attacks, \
from a total of {} interactions".format(normal_raw_data_count,attack_raw_data_count,raw_data_count))

There are 97278 normal interactions and 396743 attacks, from a total of 494021 interactions


だから今私たちは2つのRDDを持っています.1つは正常なやりとりと、もう1つは攻撃です。

## デカルトを使用したプロトコルとサービスの組み合わせ

デカルト変換を使用して、2つのRDD間のデカルト積を計算することができます。 これは、2つのRDDの間にあるすべての可能な要素のペアを返します。 私たちの場合、私たちはネットワークインタラクションのサービスとプロトコルの間のすべての可能な組み合わせを生成するためにそれを使用します。

まず、2つの別々のRDDで各値の集合を分離する必要があります。 そのためには、CSVで解析されたデータセットで `distinct`を使用します。 [データセットの説明]（http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names）から、プロトコルは2番目の列であり、サービスは3番目であることがわかります（タグは最初のものではなく最後のものです このページにあるように）。

だから最初に、プロトコルを手に入れましょう。

In [11]:
csv_data = raw_data.map(lambda x: x.split(","))
protocols = csv_data.map(lambda x: x[1]).distinct()
protocols.collect()

['tcp', 'udp', 'icmp']

今我々はサービスのために同じことをする。

In [12]:
services = csv_data.map(lambda x: x[2]).distinct()
services.collect()

['http',
 'smtp',
 'finger',
 'domain_u',
 'auth',
 'telnet',
 'ftp',
 'eco_i',
 'ntp_u',
 'ecr_i',
 'other',
 'private',
 'pop_3',
 'ftp_data',
 'rje',
 'time',
 'mtp',
 'link',
 'remote_job',
 'gopher',
 'ssh',
 'name',
 'whois',
 'domain',
 'login',
 'imap4',
 'daytime',
 'ctf',
 'nntp',
 'shell',
 'IRC',
 'nnsp',
 'http_443',
 'exec',
 'printer',
 'efs',
 'courier',
 'uucp',
 'klogin',
 'kshell',
 'echo',
 'discard',
 'systat',
 'supdup',
 'iso_tsap',
 'hostnames',
 'csnet_ns',
 'pop_2',
 'sunrpc',
 'uucp_path',
 'netbios_ns',
 'netbios_ssn',
 'netbios_dgm',
 'sql_net',
 'vmnet',
 'bgp',
 'Z39_50',
 'ldap',
 'netstat',
 'urh_i',
 'X11',
 'urp_i',
 'pm_dump',
 'tftp_u',
 'tim_i',
 'red_i']

この場合は長いリストです。

今、我々はデカルト積（直積集合）を行うことができます。

In [11]:
product = protocols.cartesian(services).collect()
print "There are {} combinations of protocol X service".format(len(product))

There are 198 combinations of protocol X service


明らかに、このような小さなRDDでは、Sparkデカルト積を使用するのは実際には意味がありません。 我々はdistinctを使った後に値を完全に収集し、デカルト積を局所的に行うことができた。 さらに、個別およびデカルトは高価な操作であるため、操作データセットが大きい場合には注意して使用する必要があります。